In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import datetime

In [ ]:
import importlib
import evotsc
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_plot)

In [ ]:
## Constants
# Population
nb_indivs = 100
nb_genes = 60

# Genome
intergene = 500
gene_length = 1000
interaction_dist = 5000
interaction_coef = 0.3
sigma_basal = -0.066
sigma_opt = -0.042
epsilon = 0.005
m = 2.5
default_basal_expression = (1 + np.exp(- m)) / 2

# Fitness
selection_coef = 50

# Selection
selection_method = 'fit-prop'

# Environment
sigma_A = 0.01
sigma_B = -0.01

# Mutations
inversion_poisson_lam = 2.0
intergene_poisson_lam = 0.0 #2.0
intergene_mutation_var = 0.0 #1e1
basal_sc_mutation_prob = 0.0 #1e-1
basal_sc_mutation_var = 0.0 #1e-4

In [ ]:
rng = np.random.default_rng(seed=424242)

In [ ]:
init_genes = evotsc.Gene.generate(intergene=intergene,
                                  length=gene_length,
                                  nb_genes=nb_genes,
                                  default_basal_expression=default_basal_expression,
                                  rng=rng)

In [ ]:
init_indiv = evotsc.Individual(genes=init_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=interaction_coef,
                               sigma_basal=sigma_basal,
                               sigma_opt=sigma_opt,
                               epsilon=epsilon,
                               m=m,
                               selection_coef=selection_coef,
                               rng=rng)

In [ ]:
mutation = evotsc.Mutation(basal_sc_mutation_prob=basal_sc_mutation_prob,
                           basal_sc_mutation_var=basal_sc_mutation_var,
                           intergene_poisson_lam=intergene_poisson_lam,
                           intergene_mutation_var=intergene_mutation_var,
                           inversion_poisson_lam=inversion_poisson_lam)

In [ ]:
evotsc_plot.plot_expr_AB(indiv=init_indiv, sigma_A=sigma_A, sigma_B=sigma_B,
                         plot_title='best generation 0', plot_name=f'best generation {0:05}.pdf')
evotsc_plot.explain(indiv=init_indiv, sigma_A=sigma_A, sigma_B=sigma_B)
evotsc_plot.plot_genome_and_tsc(init_indiv, print_ids=True, sigma=sigma_B, name=f'best genome gen {0:05}.pdf')

In [ ]:
population = evotsc.Population(init_indiv=init_indiv,
                               nb_indivs=nb_indivs,
                               mutation=mutation,
                               sigma_A=sigma_A,
                               sigma_B=sigma_B,
                               selection_method=selection_method,
                               rng=rng)
population.evaluate()

In [ ]:
step = 50

In [ ]:
for gen in range(1001):
    best_indiv, avg_fit = population.step()
    
    if gen % 10 == 0:
        time = datetime.datetime.now().strftime("%H:%M:%S")
        print(f'Generation {gen:05} ({time}): best fit {best_indiv.fitness:.5}, avg fit {avg_fit:.5}')
    
    if gen % step == 0:
        #evotsc_plot.plot_expr_AB(indiv=best_indiv, sigma_A=sigma_A, sigma_B=sigma_B,
                                 #plot_title=f'best generation {gen:05}',
                                 #plot_name=f'best generation {gen:05}.pdf')
        evotsc_plot.explain(best_indiv, sigma_A=sigma_A, sigma_B=sigma_B)
        #evotsc_plot.plot_genome(best_indiv, name=f'best genome gen {gen:05}.png')
        print()

In [ ]:
last_indiv, avg = population.step()

In [ ]:
evotsc_plot.explain(last_indiv, sigma_A=sigma_A, sigma_B=sigma_B)

In [ ]:
def run(pop, nb_steps):
    for i in range(nb_steps):
        if i % 10 == 0:
            print(f'Generation {i}')
        pop.step()